## Pipeline Steps

#### Pre-Step: Calibrate the camera to get the camera matrix and distortion coefficients
 
#### Pipeline
 1. Undistort the image
 2. Threshold the image using gradients and colors
 3. Apply a Perspective Transform to view the image from top-down
 4. Identify the lane lines
 5. Un-transform back to orginal perspective
 6. Draw lane information on original image

### Pre-Step: Calibrate the Camera

In [53]:
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

im_paths = glob.glob('./camera_cal_images/calibration*.jpg')

In [57]:
cb_shape = (9, 6)  # Corners we expect to be detected on the chessboard

obj_points = []  # 3D points in real-world space
img_points = []  # 2D points in the image

for im_path in im_paths:
    img = mpimg.imread(im_path)
    print(im_path, img.shape)
    
    obj_p = np.zeros((cb_shape[0]*cb_shape[1], 3), np.float32)
    coords = np.mgrid[0:cb_shape[0], 0:cb_shape[1]].T.reshape(-1, 2)  # x, y coords

    obj_p[:,:2] = coords
    obj_points.append(obj_p)

    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    found_all, corners = cv2.findChessboardCorners(gray, cb_shape, None)
    if found_all:
        img_points.append(corners)
        img = cv2.drawChessboardCorners(img, cb_shape, corners, found_all)
    else:
        print("Couldn't find all corners in image:", im_path)

./camera_cal_images/calibration10.jpg (720, 1280, 3)
./camera_cal_images/calibration11.jpg (720, 1280, 3)
./camera_cal_images/calibration12.jpg (720, 1280, 3)
./camera_cal_images/calibration13.jpg (720, 1280, 3)
./camera_cal_images/calibration14.jpg (720, 1280, 3)
./camera_cal_images/calibration15.jpg (720, 1280, 3)
./camera_cal_images/calibration16.jpg (720, 1280, 3)
./camera_cal_images/calibration17.jpg (720, 1280, 3)
./camera_cal_images/calibration18.jpg (720, 1280, 3)
./camera_cal_images/calibration19.jpg (720, 1280, 3)
./camera_cal_images/calibration2.jpg (720, 1280, 3)
./camera_cal_images/calibration20.jpg (720, 1280, 3)
./camera_cal_images/calibration3.jpg (720, 1280, 3)
./camera_cal_images/calibration6.jpg (720, 1280, 3)
./camera_cal_images/calibration7.jpg (720, 1280, 3)
./camera_cal_images/calibration8.jpg (720, 1280, 3)
./camera_cal_images/calibration9.jpg (720, 1280, 3)
